In [ ]:
# We use the simplextree library
# from https://github.com/peekxc/simplextree-py/tree/main
# Tested with symplextree 0.2.1
#!pip install simplextree


# Computing gradient vector fields with Morse sequences
### G. Bertrand and L. Najman

This notebook contains a pure python implementation of the algorithms proposed in the paper **XXX**. 

As it is in python, the implementation is not as efficient as a C/C++ one, but it works.

For manipulationg the simplicial complexes, we use the simplex tree package from https://github.com/peekxc/simplextree-py/tree/main.

Latex macros
$\newcommand{\ms}{\overrightarrow{W}}$
$\newcommand{\ka}{\kappa}$

# Morse sequences

Let $K$ be a finite family composed
of non-empty finite sets, called *simplexes*.
The family $K$ is a  *(simplicial) complex* if $\sigma \in K$ whenever $\sigma \not= \emptyset$ and $\sigma \subseteq \tau$ for some $\tau \in K$. 
An element of a simplicial complex $K$ is a *face of~$K$*.
A *facet of~$K$* is a face of $K$ that is maximal for inclusion.
The *dimension* of $\sigma \in K$, written $dim(\sigma)$,
is the number of its elements
minus one. If $\sigma \in K$, we write: 
-  $\partial (\sigma) = \{ \nu \in K \; | \; \nu \subseteq \sigma$ and $dim(\nu) =dim(\sigma)-1 \}$, and
- $\delta (\sigma) = \{ \nu \in K \; | \; \sigma \subseteq \nu$ and $dim(\nu) =dim(\sigma)+1 \}$.

$\partial (\sigma)$ and $\delta (\sigma)$ are, respectively, the *boundary* and the *coboundary* of $\sigma$ in $K$.

A  *subcomplex of $K$* is a set $L \subseteq K$ which is a simplicial complex.  
We recall the definitions of simplicial collapses and simplicial expansions.

Let $K$ be a simplicial complex and let $\sigma, \tau \in K$.  
The couple $(\sigma,\tau)$ is a *free pair for $K$*, if $\tau$ is the only face of $K$ that contains $\sigma$.  
If $(\sigma,\tau)$ is a free pair for $K$, then the simplicial complex $L = K \setminus \{ \sigma,\tau \}$
is an *elementary collapse of $K$*, and $K$ is an *elementary expansion of $L$*. 
We say that $K$ *collapses onto $L$*, or that $L$ *expands onto $K$*, if there exists a  sequence $\langle K=K_0,\ldots,K_k=L \rangle$, such that $K_i$ is an elementary collapse of $K_{i-1}$, $i \in [1,k]$.


We also recall  the definitions of perforations and fillings. 
Let $K,L$ be simplicial complexes.
If $\nu \in K$ is a facet of $K$ and if $L = K \setminus \{\nu \}$, we say that
$L$ is an *elementary perforation of $K$*, and that $K$ is an *elementary filling of $L$*.


We now introduce the notion of a ``Morse sequence'' by simply considering expansions and fillings
of a simplicial complex.


**Definition** Let $L\subseteq K$ be two simplicial complexes. A *Morse sequence from $L$ to $K$* is a sequence
$\ms = \langle L = K_0,\ldots,K_k =K \rangle$ of simplicial complexes
such that, for each $i \in [1,k]$, $K_i$ is either an elementary expansion or an elementary filling of $K_{i-1}$.  
If $L=\emptyset$, we say that $\ms$ is a *Morse sequence on $K$*.

Thus, any Morse sequence $\ms$ on $K$ is a *filtration on $K$*, that is a sequence of nested complexes  $\langle \emptyset = K_0,...,K_k =K \rangle$
such that, for each $i \in [0,k-1]$, we have $K_i \subseteq K_{i+1}$

Let $\ms = \langle K_0,\ldots,K_k \rangle$ be a Morse sequence.
For each $i \in [1,k]$:  
- If $K_i$ is an elementary filling of $K_{i-1}$, we write $\kappa_i$ for the simplex such that $K_i = K_{i-1} \cup \{\kappa_i\}$. We say that
the face $\kappa_i$ is *critical for $\ms$*. 
- If $K_i$ is an elementary expansion of $K_{i-1}$, we write
$\kappa_i$  for the free pair $(\sigma,\tau)$ such that $K_i = K_{i-1} \cup \{\sigma,\tau \}$. We say that
$\kappa_i$, $\sigma$, $\tau$, are *regular for $\ms$*. 

**Definition** Let $\ms$ be a Morse sequence. 
The *gradient vector field of $\ms$* is the set composed of all regular pairs for $\ms$.


The two following schemes are two basic ways to build a Morse sequence $\ms$ from $L$ to $K$.
1. *The increasing scheme*. We build $\ms$ from the left to the right. Starting from $L$, we obtain $K$ by iterative expansions and fillings. We say that this scheme is *maximal* if we make a filling only if no expansion can be made. 
2. *The decreasing scheme*. We build $\ms$ from the right to the left. Starting from $K$, we obtain $L$ by iterative collapses and perforations. We say that this scheme is *minimal* if we make a perforation only if no collapse can be made. 



# Cosimplicial complex
**Definition** Let $S$ be a finite set of simplexes. The set $S$ is a *cosimplicial complex* if,
for any $\nu,\mu \in S$, we have $\eta \in S$ whenever $\nu \subseteq \eta \subseteq \mu$. 

Let $S$ be a finite set of simplexes. 
Let $\overline{S}$ be the set of simplexes such that
$\nu \in \overline{S}$ if and only if there exists $\mu \in S$ with $\nu \subseteq \mu$. 
Thus, we have $S \subseteq \overline{S}$. We observe that
$\overline{S}$ is a simplicial complex, this complex is the smallest simplicial complex that contains $S$.
It follows that $S$ is a simplicial complex if and only if $\overline{S} = S$.

We write $\underline{S} = \overline{S} \setminus S$.

Let $S$ be a cosimplicial complex. Let $\partial$ and $\delta$ be, respectively, the boundary and the coboundary operators relative to the simplicial complex $\overline{S}$.
For each $\sigma \in S$, we write $\partial(\sigma,S) = \partial(\sigma) \cap S$ and $\delta(\sigma,S) = \delta(\sigma)$.

- the complex $S \setminus \{ \sigma, \tau \}$ is a *reduction of $S$* if $\delta(\sigma,S) = \{ \tau \}$,
- the complex $S \setminus \{ \nu \}$ is a *perforation of $S$* if $\delta(\nu,S) = \emptyset$,
- the complex $S \setminus \{ \sigma, \tau \}$ is a *coreduction of $S$* if $\partial(\tau,S) = \{ \sigma \}$, 
- the complex $S \setminus \{ \nu \}$ is a *coperforation of $S$* if $\partial(\nu,S) = \emptyset$.


# $F$-sequences
Let $F$ be a map from a cosimplicial complex $S$ to $\mathbb{Z}$. We say that $F$ is a *stack on $S$* if
we have $F(\sigma) \leq F(\tau)$ whenever $\sigma,\tau \in S$ and $\sigma\subseteq \tau$. 

Let $F$ be a map from a cosimplicial complex $S$ to $\mathbb{Z}$.
For any $\lambda \in \mathbb{Z}$, we write:  
- $\overline{F}[\lambda] = \{\sigma \in S \; \mid \; F(\sigma) \leq \lambda \}$ and
- $F[\lambda] = \{\sigma \in S \; \mid \; F(\sigma) = \lambda \}$,  
$\overline{F}[\lambda]$ and $F[\lambda]$ are, respectively, the *(lower) cut* and the *section of $F$ at level $\lambda$*.

Remark that if $K$ is a simplicial complex, then, as, for any $\lambda\in\mathbb{Z}$, we have $\overline{F}[\lambda]\subseteq \overline{F}[\lambda+1]$, the indexed family $(\overline{F}[\lambda])_{\lambda\in\mathbb{Z}}$ is a filtration on $K$.

**In the sequel of this notebook, $L$ and $K$ will denote simplicial complexes.**

**Definition** Let $F$ be a stack on $K$ and let $L$ be a subcomplex of $K$. Let $(\sigma,\tau)$ be a free pair for $L$.
We say that $(\sigma,\tau)$ is 
a *free pair for $F$* if $F(\sigma) = F(\tau)$. If $\kappa = (\sigma,\tau)$ is a free pair for $F$, we say that
$L' = L \setminus \{\sigma,\tau \}$ is an *(elementary) $F$-collapse of $L$*
and $L$ is an *(elementary) $F$-expansion of $L'$*. 
We write $F(\kappa) = F(\sigma) = F(\tau)$.

Let $F$ be a stack on $K$ and let $L$ be a subcomplex of $K$.  If $\lambda \in \mathbb{Z}$, we write $\overline{L}[\lambda] = \overline{F}[\lambda] \cap L$.  
The set $\overline{L}[\lambda]$ is the *section of $L$ (for $F$) at level $\lambda$*.

**Definition** Let $F$ be a stack on $K$ and $\ms$ be a Morse sequence from $L$ to $K$.  
We say that $\ms$ is an *$F$-sequence*, if each regular pair for $\ms$ is a free pair for £$F$.



# Maximal and minimal $F$-sequences

Let $F$ be a stack on $K$ and let $\ms = \langle L = K_0,..., K_k = K \rangle$
be a Morse sequence which is an $F$-sequence.  
For any $i \in [0,k]$, we say that $K_i$ is *maximal for $F$* (resp. *minimal for $F$*) if
no $F$-expansion (resp. $F$-collapse) of $K_i$ is a subset of $K$ (resp includes $L$).  
We say that $\ms$ or $\diamond \ms$ is *maximal  for $F$* if, for any $i \in [1,k]$, the complex $X_{i-1}$ is maximal  for $F$ whenever $X_i$ is critical for $\ms$.  
We say that $\ms$ or $\diamond \ms$ is *minimal  for $F$* if, for any $i \in [0,k-1]$, the complex $X_{i+1}$ is minimal  for $F$ whenever $X_i$ is critical for $\ms$. 


# Implementation of the algorithms

In [2]:
from simplextree import SimplexTree
from itertools import combinations

## Helper functions
The following are some helper functions computing boundary, coboundary, and efficiently computing the difference of two lists.

In [3]:
def boundary(sigma, S):
    if len(sigma) > 1:
        return [tuple(s) for s in combinations(sigma,len(sigma)-1) if S[s]]
    return list()

def coboundary(st, sigma, S): 
    return [s for s in st.cofaces(sigma) if (len(s) == len(sigma) + 1) and S[s]]

def nbcoboundary(st, sigma, S): 
    return len(coboundary(st, sigma, S))

def nbboundary(st, sigma, S): 
    return len(boundary(sigma, S))

def difflist(list1, list2): 
    # Can be used to obtain the differences of two simplicial complexes
    ## st1 = SimplexTree([[0,1,2]])
    ## st2 = SimplexTree([[1,2]])
    ## S = [[0], [0,1], [0,2], [0,1,2]]
    ## assert S == difflist(st1.simplices(), st2.simplices())

    s1 = {tuple(i) for i in list1}
    s2 = {tuple(i) for i in list2}
    l = list(list(i) for i in (s1 - s2))
    return sorted(l, key=len)

## Maximal increasing Morse sequence

**Data**
- A cosimplicial complex $S$ with its operators $\partial$ and $\delta$; and
- A stack $F : S \rightarrow \mathbb{Z}$.

The datastructure for $S$ is an array that stores the simplexes according to their increasing dimension and weight: we have, for $1 \leq i < j \leq N = Card(S)$:
- $dim(S[i]) \leq dim(S[j])$ whenever $F(S[i]) = F(S[j])$; and 
- $F[S[i]] < F(S[j])$ otherwise.

**Result**
A Morse sequence from $\underline{S}$ to $\overline{S}$ which is maximal for $F$.

In [4]:
# computes a maximal increasing Morse sequence obtained 
# from a cosimplicial complex S weighted by a function F.
def Max(S, st, F):
    T = dict()
    Sdict = dict()
    U = list()
    MorseSequence=list()
    N = len(S)

    for s in st.simplices():
        T[s] = False
        Sdict[s] = False

    rho = dict()
    for s in S:
        Sdict[s] = True
        nb = nbboundary(st, s, Sdict)
        rho[s] = nb
        if nb == 1:
             U.append(s)

    i = 0
    while i<N:
        while U:
            tau = U.pop(0)
            if rho[tau] == 1:
                sigma = next(s for s in boundary(tau, Sdict) if not T[s])
                if F[sigma] == F[tau]:
                    MorseSequence.append([sigma, tau])
                    T[tau] = True
                    T[sigma] = True
                    for mu in coboundary(st, sigma, Sdict)+coboundary(st, tau, Sdict):
                        rho[mu] = rho[mu] - 1
                        if rho[mu] == 1:
                             U.append(mu)
        while i<N and T[S[i]]:
            i += 1
        if i<N:
            sigma = S[i]
            MorseSequence.append([sigma])
            T[sigma] = True
            for tau in coboundary(st, sigma, Sdict):
                    rho[tau] = rho[tau] - 1
                    if rho[tau] == 1:
                        U.append(tau)
    return MorseSequence


st = SimplexTree([[0,1,2]])
S = sorted(st.simplices(), key=lambda x: (len(x), x))
F = dict()
for s in S:
    F[s] = 0
F[(0, 1, 2)] = 1
Max(S, st, F)

[[(0,)], [(1,), (0, 1)], [(2,), (0, 2)], [(1, 2)], [(0, 1, 2)]]

## Minimal decreasing Morse sequence

**Data**  

- A cosimplicial complex $S$ with its operators $\partial$ and $\delta$; and
- A stack $F : S \rightarrow \mathbb{Z}$.

The datastructure for $S$ is an array that stores the simplexes according to their decreasing dimension and weight: we have, for $1 \leq i < j \leq N = Card(S)$: 
- $dim(S[i]) \geq dim(S[j])$ whenever $F(S[i]) = F(S[j])$; and
- $F[S[i]] > F(S[j])$ otherwise.

**Result**
A Morse sequence from $\underline{S}$ to $\overline{S}$ which is minimal for $F$.


In [5]:
# computes a minimal increasing Morse sequence obtained 
# from a cosimplicial complex S weighted by a function F.
def Min(S, st, F):
    T = dict()
    Sdict = dict()
    U = list()
    MorseSequence=list()
    N = len(S)

    for s in st.simplices():
        T[s] = False
        Sdict[s] = False

    rho = dict()
    for s in S:
        Sdict[s] = True
        nb = nbcoboundary(st, s, Sdict)
        rho[s] = nb
        if nb == 1:
             U.append(s)

    i = 0
    while i<N:
        while U:
            tau = U.pop(0)
            if rho[tau] == 1:
                sigma = next(s for s in coboundary(st, tau, Sdict) if not T[s])
                if F[sigma] == F[tau]:
                    MorseSequence.append([tau, sigma])
                    T[tau] = True
                    T[sigma] = True
                    for mu in boundary(sigma, Sdict)+boundary(tau, Sdict):
                        rho[mu] = rho[mu] - 1
                        if rho[mu] == 1:
                             U.append(mu)
        while i<N and T[S[i]]:
            i += 1
        if i<N:
            sigma = S[i]
            MorseSequence.append([sigma])
            T[sigma] = True
            for tau in boundary(sigma, Sdict):
                    rho[tau] = rho[tau] - 1
                    if rho[tau] == 1:
                        U.append(tau)
    return MorseSequence[::-1]

st = SimplexTree([[0,1,2]])
S = sorted(st.simplices(), key=lambda x: (len(x), x))[::-1]
F = dict()
for s in S:
    F[s] = 0
F[(0, 1, 2)] = 0
Min(S, st, F)

[[(0,)], [(2,), (0, 2)], [(1,), (0, 1)], [(1, 2), (0, 1, 2)]]

# Figures from the paper

In [6]:
# From https://stackoverflow.com/questions/44934631/making-grid-triangular-mesh-quickly-with-numpy
def MakeFacesVectorized1(Nr,Nc):

    out = np.empty((Nr-1,Nc-1,2,3),dtype=int)

    r = np.arange(Nr*Nc).reshape(Nr,Nc)

    out[:,:, 0,0] = r[:-1,:-1]
    out[:,:, 1,0] = r[:-1,1:]
    out[:,:, 0,1] = r[:-1,1:]

    out[:,:, 1,1] = r[1:,1:]
    out[:,:, :,2] = r[1:,:-1,None]

    out.shape =(-1,3)
    return out

import numpy as np

In [10]:
listOfTriangles = MakeFacesVectorized1(5,5)
x = np.linspace(0, 4, 5)
y = np.linspace(0, 4, 5)
xv, yv = np.meshgrid(x, y)

st = SimplexTree(listOfTriangles)
F=dict()
for i in st.simplices():
    F[i]=1
F[(6,)]=0
F[(18,)]=0
F[(16, 17, 21)]=3
F[(3, 7, 8)]=2
F[(7, 8)]=F[(3, 8)]=2
F[(3, 4, 8)]=2
F[(8,)]=2
F[(4, 8)]=2
F[(7, 11, 12)]=F[(7, 12)]=F[(11, 12)]=2
F[(7, 8, 12)]=F[(8, 12)]=2
F[(8, 12, 13)]=F[(8, 13)]=2
F[(8, 9, 13)]=3
F[(11, 12, 16)]=F[(11, 15, 16)]=F[(12, 16)]=F[(11, 16)]=2
F[(12, 13, 17)]=F[(12, 16, 17)]=F[(12, 17)]=2
F[(15, 16, 20)]=F[(16, 20)]=F[(15, 20)]=F[(15, 16)]=F[(16, 17)]=2
F[(16, 20, 21)]=F[(16, 21)]=F[(16,)]=2
F[(4, 8, 9)]=F[(8, 9)]=F[(4, 9)]=3

S = sorted(st.simplices(), key=lambda s: (F[s], len(s)))
msMax = Max(S, st, F)
S = sorted(st.simplices(), key=lambda s: (-F[s], -len(s)))
msMin = Min(S, st, F)

# Maximal sequences and vertex maps
A *vertex map on $K$* is a map  $f : V(K) \rightarrow \mathbb{Z}$,
where $V(K)$ is the vertex set of $K$, that is, the set composed of all simplexes $\sigma \in K$ such that 
$dim(\sigma) = 0$. 

Let $f$ be a vertex map on $K$. The map $f$ may be extended to all simplexes of $K$ as follows. 
For any $\tau \in K$, we write:  

 $F(\tau) = \max \{ f(\sigma) \; | \; \sigma \in V(K)$ and $\sigma \subseteq \tau \}$. 

Clearly, the map $F$ is a stack on $K$. We say that $F$ is the *stack induced by $f$*. 

We say that a vertex map $f$ on $K$ is a *$\vartheta$-map* if $f$ maps distinct elements to distinct elements, that is, if $f$ is injective. If $f$ is a $\vartheta$-map, we say that the stack induced by $f$ is a *$\vartheta$-stack*. 
Also, if $\sigma_1,...,\sigma_N$ is a total ordering of the vertices of $K$, we say that this ordering is 
the *ordering induced by $f$}* if we have $f(\sigma_i) < f(\sigma_j)$ whenever $i < j$. 


Let $F$ be a $\vartheta$-stack on $K$. 
If $\sigma \in V(K)$,
the *lower star of $\sigma$* is the set:  
$\hat{\delta}(\sigma) = \{\tau \in K \; | \; \sigma \subseteq \tau$ and $F(\tau) = F(\sigma) \}$. 

An $F$-sequence may be obtained by considering each lower star independently.
Since the values of $F$ on a lower star are equal, we introduce the following.

If $S$ is a set, we write   $\mathbb{1}_S$ for the function $S \rightarrow \mathbb{Z}$
such that, for each $x \in S$, we have $\mathbb{1}_S(x) = 1$. This $\mathbb{1}_S$ is a constant function.

If $\ms$ is a Morse sequence on $K$, we say that $\ms$  is *maximal* if $\ms$ is maximal for $\mathbb{1}_K$. 


The algorithm below allows to compute an $F$-sequence, where $F$ is a stack on~$K$ induced by a $\vartheta$-map $f$. It uses a function ${\bf Max}(S)$ which is a particular case of the function  ${\bf Max}(S,F)$
introduced above. More precisely, we have   ${\bf Max}(S) = {\bf Max}(S,\mathbb{1}_S)$. 


In [8]:
def MaxParallel(lowerstar, st, F):
    MS = list()
    for i in range(len(lowerstar)): # Shall be done in parallel
       MS.append(Max(lowerstar[i][1], st, F))
    MorseSequence=list()
    for i in range(len(MS)):
        MorseSequence.extend(MS[i])
    return MorseSequence

st = SimplexTree([[0,1,2]])
S = st.simplices(0)
F = dict()
# Initialize the function F on the vertices
weight = np.random.randint(0, 10, len(S)) # Random weights
weight = np.argsort(np.argsort(weight)) # Sort the weights, and replace them by their rank
i = 0
for s in S:
    F[s] = weight[i]
    i += 1
# Initialize the function F on the rest of the simplices, by order of dimension
i = 0
# Build the simplicial stack
for d in st.n_simplices:
    if i != 0:
        for sigma in st.simplices(i):
            max = 0
            b = [tuple(s) for s in combinations(sigma,len(sigma)-1)]
            for tau in b:
                if F[tau] > max:
                    max = F[tau]
            F[sigma] = max
    i += 1
# Now we have a simplicial stack, we can compute the lower star

# First, inverse the function F
# We need to create a dictionary with the values of F as keys
# and the simplices as values
lowerstar =dict()
for key, value in F.items():
    lowerstar.setdefault(value, []).append(key)
lowerstar = sorted(lowerstar.items())
ms=MaxParallel(lowerstar, st, F)
print("The stack is:", F)
print("The lower star is:", lowerstar)
print("The Morse sequence is", ms)

The stack is: {(0,): 1, (1,): 2, (2,): 0, (0, 1): 2, (0, 2): 1, (1, 2): 2, (0, 1, 2): 2}
The lower star is: [(0, [(2,)]), (1, [(0,), (0, 2)]), (2, [(1,), (0, 1), (1, 2), (0, 1, 2)])]
The Morse sequence is [[(2,)], [(0,), (0, 2)], [(1,), (0, 1)], [(1, 2), (0, 1, 2)]]
